In [92]:
run ~/w/replab/replab_init
replab.globals.verbosity(0);
replab.globals.useReconstruction(1);

In this notebook, we compute the family of states with the symmetry of the GHZ states, as in [C. Eltschka and J. Siewert, "Entanglement of Three-Qubit Greenberger-Horne-Zeilinger--Symmetric States"](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.108.020502).


The GHZ state $\left|\text{GHZ}\right> = \left|000\right> + \left|111\right>$ is invariant (U |GHZ> = |GHZ>) under a family of matrices U

\begin{equation}
U = \begin{pmatrix} a_0 & 0 \\ 0 & a_1 \end{pmatrix} \otimes \begin{pmatrix} b_0 & 0 \\ 0 & b_1 \end{pmatrix} \otimes \begin{pmatrix} c_0 & 0 \\ 0 & c_1 \end{pmatrix} 
\end{equation}

who act on the state space $\mathbb{C}^8$. The eight coefficients, in order, are those of $\left|000\right>$,$\left|001\right>$,$\left|010\right>$,$\left|011\right>$,$\left|100\right>$,$\left|101\right>$,$\left|110\right>$,$\left|111\right>$.

For the invariance to hold, the unit complex numbers $a_0$,$a_1$,$b_0$,$b_1$,$c_0$,$c_1$ obey the equation $a_0 b_0 c_0 = a_1 b_1 c_1 = 1$.

The state is also invariant under permutation of subsystems (a group of order $3! = 6$), and permutation of the levels (a group of order $2$).

Let us define the continuous connected group. $T(6)$ is the [torus group](https://en.wikipedia.org/wiki/Torus#n-dimensional_torus) with 6 elements, elements that we name according to our scenario.

In [97]:
T6 = replab.T(6);
T6 = T6.withNames({'a0' 'b0' 'c0' 'a1' 'b1' 'c1'});

We then define the subgroup obeying the equation $a_0 b_0 c_0 = a_1 b_1 c_1 = 1$.

In [98]:
T = T6.subgroupWith('a0*b0*c0 = 1', 'a1*b1*c1 = 1')

T = 
Torus group of dimension n=6 and rank r=4
   identity: [0; 0; 0; 0; 0; 0]
  injection: 6 x 4 double
      names: {'a0', 'b0', 'c0', 'a1', 'b1', 'c1'}
 projection: 4 x 6 double
equation(1): 'a0 b0 c0'
equation(2): 'a1 b1 c1'


Now, how does that group act on the state space $\mathbb{C}^8$?

We construct the representation whose image is the matrix $U$ above.

In [100]:
Trep = T.diagonalRepWith('a0 b0 c0', ...
                         'a0 b0 c1', ...
                         'a0 b1 c0', ...
                         'a0 b1 c1', ...
                         'a1 b0 c0', ...
                         'a1 b0 c1', ...
                         'a1 b1 c0', ...
                         'a1 b1 c1');

We construct now the discrete part, by writing how the subsystem and level permutations affect the elements of the continuous connected part.

This finite group permutes the three subsystems and the two levels, independently, so we write it as a direct product.

In [101]:
S3 = replab.S(3);
S2 = replab.S(2);
F = S3.directProduct(S2)

F = 
Direct product group with 2 factors of order 12
    identity: {[1, 2, 3], [1, 2]}
        type: Direct product group with 2 factors of order 12
   factor(1): Symmetric group acting on 3 elements
   factor(2): Symmetric group acting on 2 elements
generator(1): {[2, 3, 1], [1, 2]}
generator(2): {[2, 1, 3], [1, 2]}
generator(3): {[1, 2, 3], [2, 1]}
   recognize: AtlasResult (Dihedral group of order 12)


Now, we write the action of the generators of this discrete group on the torus elements:

In [102]:
% Permutation of AB
gAB = {[2 1 3] [1 2]};
actAB = T.automorphism('b0', 'a0', 'c0', 'b1', 'a1', 'c1');
% Permutation of AC
gAC = {[3 2 1] [1 2]};
actAC = T.automorphism('c0', 'b0', 'a0', 'c1', 'b1', 'a1');
% Permutation of BC
gBC = {[1 3 2] [1 2]};
actBC = T.automorphism('a0', 'c0', 'b0', 'a1', 'c1', 'b1');
% Permutation of the two levels
gL = {[1 2 3] [2 1]};
actL = T.automorphism('a1', 'b1', 'c1', 'a0', 'b0', 'c0');

We check that the generators generate the whole group.

In [104]:
assert(F.subgroup({gAB, gAC, gBC, gL}) == F);

The [outer semidirect product construction](https://en.wikipedia.org/wiki/Semidirect_product#Inner_and_outer_semidirect_products) employs a morphism from a group $H$ to the automorphisms of another gorup $N$. This is what we do now.

In [107]:
G = T.semidirectProductByFiniteGroup(F, 'preimages', {gAB, gAC, gBC, gL}, 'images', {actAB, actAC, actBC, actL});

## How does this group act on the state space?
Remember that the integers $1$, $2$, $3$, $4$, $5$, $6$, $7$, $8$ enumerate $\left|000\right>$, $\left|001\right>$, $\left|010\right>$, $\left|011\right>$, $\left|100\right>$, $\left|101\right>$, $\left|110\right>$, $\left|111\right>$.

In [108]:
imgAB = replab.Permutation.toMatrix([1 2 5 6 3 4 7 8]);
imgAC = replab.Permutation.toMatrix([1 5 3 7 2 6 4 8]);
imgBC = replab.Permutation.toMatrix([1 3 2 4 5 7 6 8]);
imgL = replab.Permutation.toMatrix([8 7 6 5 4 3 2 1]);

We now construct the representation of the discrete group on the state space.

In [109]:
Frep = F.repByImages('C', 8, 'preimages', {gAB, gAC, gBC, gL}, 'images', {imgAB, imgAC, imgBC, imgL});

We had the representation of the torus group `Trep`, and we now assemble the two.

In [110]:
rep = G.semidirectProductRep(Frep, Trep)

rep = 
Unitary representation
               Hrep: Unitary representation
               Nrep: Unitary representation
          dimension: 8
divisionAlgebraName: []
              field: 'C'
              group: replab.prods.SemidirectProductGroup_compact
          isUnitary: true


... and now it is time to decompose that decomposition.

In [111]:
rep.decomposition

ans = 
Unitary reducible representation
          dimension: 8
divisionAlgebraName: []
              field: 'C'
              group: replab.prods.SemidirectProductGroup_compact
 injection_internal: 8 x 8 double
       isSimilarRep: true
          isUnitary: true
     mapsAreAdjoint: true
             parent: Unitary representation
projection_internal: 8 x 8 double
  basis(1,'double'): [0.70711; 0; 0; 0; 0; 0; 0; 0.70711]
  basis(2,'double'): [0.70711; 0; 0; 0; 0; 0; 0; -0.70711]
  basis(3,'double'): [0; 1; 0; 0; 0; 0; 0; 0]
  basis(4,'double'): [0; 0; 1; 0; 0; 0; 0; 0]
  basis(5,'double'): [0; 0; 0; 1; 0; 0; 0; 0]
  basis(6,'double'): [0; 0; 0; 0; 1; 0; 0; 0]
  basis(7,'double'): [0; 0; 0; 0; 0; 1; 0; 0]
  basis(8,'double'): [0; 0; 0; 0; 0; 0; 1; 0]
       component(1): Isotypic component C(1) (trivial)
       component(2): Isotypic component C(1) (nontrivial)
       component(3): Isotypic component C(6) (nontrivial)


Thus we recover that the states invariant under this type of symmetries are the pure states $\left| \text{GHZ}_\pm \right> = \frac{\left| 000 \right> \pm \left| 111 \right>}{\sqrt{2}}$ and a mixed state containing all basis elements except $\left| 000 \right>$ and $\left| 111 \right>$ with equal weight $1/6$.